# Analyzing Mobile App Data
* We will pretend we work for a tech startup looking to build an app.
* We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that the number of users of our apps determines our revenue for any given app — the more users who see and engage with the ads, the better. 
* Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

We will start off by pulling in a sample dataset of iOS app data which can be found [here](https://dq-content.s3.amazonaws.com/350/AppleStore.csv) and one for the Google Play store which can be found [here](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)

In [1]:
from csv import reader
opened_file=open('AppleStore.csv')
read_file=reader(opened_file)
data=list(read_file)
ios_header=data[0]
ios=data[1:]

opened_file=open('googleplaystore.csv')
read_file=reader(opened_file)
data=list(read_file)
android_header=data[0]
android=data[1:]

Next we'll begin opening and exploring these two data sets. To make them easier to explore, we created a function named explore_data() that you can repeatedly use to print rows in a readable way.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
explore_data(ios, 0, 2, rows_and_columns=True)
explore_data(android, 0, 2, rows_and_columns=True)
print('iOS Column Names', ios_header)
print('\n')
print('Android Column Names', android_header)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7197
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13
iOS Column Names ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Android Column

The Google Play dataset has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion), and we can see that one of the [discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) describes an error for row 10472. We will now work to correct that.

In [3]:
print(android[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The app is missing the Category column data and a shift happens for the next few rows. We'll just go ahead and delete this entry.

In [4]:
del android[10472]

In both datasets there are duplicate entries for many apps, for example:

In [5]:
for app in android:
    name=app[0]
    if name == 'Snapchat':
        print(app)

['Snapchat', 'SOCIAL', '4.0', '17014787', 'Varies with device', '500,000,000+', 'Free', '0', 'Teen', 'Social', 'July 30, 2018', 'Varies with device', 'Varies with device']
['Snapchat', 'SOCIAL', '4.0', '17014705', 'Varies with device', '500,000,000+', 'Free', '0', 'Teen', 'Social', 'July 30, 2018', 'Varies with device', 'Varies with device']
['Snapchat', 'SOCIAL', '4.0', '17015352', 'Varies with device', '500,000,000+', 'Free', '0', 'Teen', 'Social', 'July 30, 2018', 'Varies with device', 'Varies with device']
['Snapchat', 'SOCIAL', '4.0', '17000166', 'Varies with device', '500,000,000+', 'Free', '0', 'Teen', 'Social', 'July 30, 2018', 'Varies with device', 'Varies with device']


Let's see just how many duplicate apps there are.

In [6]:
unique=[]
duplicate=[]
for app in android:
    name=app[0]
    if name in unique:
        duplicate.append(name)
    else:
        unique.append(name)
print(len(duplicate))
    

1181


We see that we have 1181 duplicate apps, this poses a problem and could cause error in our analysis, we need to delete the duplicates but we don't want to delete them at random. One solution is to use the number of reviews column which corresponds to the fourth position of each row. This is the main difference between them and should give us the most up-to-date information about each app. For example:

In [7]:
for app in android:
    name=app[0]
    if name == 'Snapchat':
        print(app[0], app[3])

Snapchat 17014787
Snapchat 17014705
Snapchat 17015352
Snapchat 17000166


In [8]:
print('reviews_max Expected Length', len(android)-1181)
reviews_max={}
for app in android:
    name=app[0]
    n_reviews=float(app[3])
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    if name not in reviews_max:
        reviews_max[name]=n_reviews
print(len(reviews_max))

reviews_max Expected Length 9659
9659


In [9]:
android_clean=[]
already_added=[]
for app in android:
    name=app[0]
    n_reviews=float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
explore_data(android_clean, 0, 2, rows_and_columns=True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9659
Number of columns: 13


We first created a dictionary where each key is a unique app name that corresponds to the highest number of reviews for that app. We then used that dictionary to remove duplicate rows.
* We start by initializing two different lists
* For every entry in our dataset
    * We isolate the name and number of reviews
    * We then append the name into our clean list if:
        * The number of reviews matches the value in our reviews_max dictionary AND 
        * If it is not in our already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

We then explored the new data set, and confirmed that the number of rows is indeed 9,659.

We don't need to remove duplicates in the iOS dataset because there are non.  See for yourself.

In [10]:
unique=[]
duplicate=[]
for app in ios:
    name=app[0]
    if name in unique:
        duplicate.append(name)
    else:
        unique.append(name)
print(len(duplicate))
    

0


Since we are catering to an English speaking market we want to remove apps with non-english titles from our dataset, such as:

In [11]:
print(ios[813][1])
print('\n')
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播


لعبة تقدر تربح DZ


We need to remove these apps while keeping ones in English, we can do this by removing any app name that has a character with an ASCII value over 127, the onl problem is there are some apps with English names that have one or two characters that fall outside that range such as:
'Instachat 😜'

In [12]:
print(ord('😜'))

128540


One way to get the information we need is to separate out any apps with more than three non_English characters. We can do that with this function.

In [13]:
def Eng(string):
    character_count=[]
    for character in string:
        if ord(character)>127:
           character_count.append(character)
        if len(character_count)>3:
            return False
    return True

In [14]:
print(Eng('Docs To Go™ Free Office Suite'))
print(Eng('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(Eng('Instachat 😜'))

True
False
True


We can now loop through our ios and android_clean datasets and separate out any non_English titles. Some will invariably slip through the cracks but not enough to hinder our analysis.

In [15]:
ios_english=[]
for app in ios:
    name=app[1]
    if Eng(name) == True:
        ios_english.append(app)
android_english=[]
for app in android_clean:
    name=app[0]
    if Eng(name) == True:
        android_english.append(app)
explore_data(ios_english, 0, 2, rows_and_columns=True)
explore_data(android_english, 0, 2, rows_and_columns=True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 6183
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9614
Number of columns: 13


We now need to isolate only the apps that are free to download and install.

In [16]:
ios_free=[]
for app in ios_english:
    price=app[4]
    if price == '0.0':
        ios_free.append(app)
android_free=[]
for app in android_english:
    price=app[7]
    if price == '0':
        android_free.append(app)
print(len(ios_free))
print(len(android_free))

3222
8864


Our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Let's inspect both datasets and look for columns to determine the most common genres in each market. We'll use this to build frequency tables for each.

In [17]:
print(android_header)
print('\n')
print(ios_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


It seems as though the Category and Genres column of the Google Play store located at second and tenth positions respectively. For the AppleStore the prime_genre column will work quite nicely, this is located in position twelfth.

Let's create a frequency table of each.

In [18]:
def freq_table(dataset, index):
    table={}
    for row in dataset:
        variable=row[index]
        if variable in table:
            table[variable]+=1
        else:
            table[variable]=1
    for variable in table:
        table[variable]=(table[variable]/len(table))*100
    return table
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
    

In [19]:
display_table(ios_free, -5)
print('\n')
display_table(android_free, -4)
print('\n')
display_table(android_free, 1)

Games : 8147.826086956522
Entertainment : 1104.3478260869565
Photo & Video : 695.6521739130435
Education : 513.0434782608695
Social Networking : 460.8695652173913
Shopping : 365.2173913043478
Utilities : 352.17391304347825
Sports : 300.0
Music : 286.95652173913044
Health & Fitness : 282.60869565217394
Productivity : 243.47826086956525
Lifestyle : 221.73913043478262
News : 186.95652173913044
Travel : 173.91304347826087
Finance : 156.52173913043478
Weather : 121.73913043478262
Food & Drink : 113.04347826086956
Reference : 78.26086956521739
Business : 73.91304347826086
Book : 60.86956521739131
Navigation : 26.08695652173913
Medical : 26.08695652173913
Catalogs : 17.391304347826086


Tools : 657.0175438596491
Entertainment : 471.9298245614035
Education : 415.7894736842105
Business : 357.01754385964915
Productivity : 302.6315789473684
Lifestyle : 302.6315789473684
Finance : 287.719298245614
Medical : 274.56140350877195
Sports : 269.2982456140351
Personalization : 257.89473684210526
Communic

The frequency tables we analyzed show that apps designed for fun dominate the Apple Store while the Google Play store has more of a balance. Now we'd like to determine which genres have the most users. To do this we will calculate the average number of installs for the Google Play store, because the Apple Store does not have this information we will use the rating_count_tot column instead.

In [20]:
table=freq_table(ios_free, -5)
for genre in table:
    total=0
    len_genre=0
    for row in ios_free:
        genre_app=row[-5]
        if genre_app == genre:
            tot=float(row[5])
            total+=tot
            len_genre+=1
    average=total/len_genre
    print(genre)
    print(average)

Social Networking
71548.34905660378
Photo & Video
28441.54375
Games
22788.6696905016
Music
57326.530303030304
Reference
74942.11111111111
Health & Fitness
23298.015384615384
Weather
52279.892857142855
Utilities
18684.456790123455
Travel
28243.8
Shopping
26919.690476190477
News
21248.023255813954
Navigation
86090.33333333333
Lifestyle
16485.764705882353
Entertainment
14029.830708661417
Food & Drink
33333.92307692308
Sports
23008.898550724636
Book
39758.5
Finance
31467.944444444445
Education
7003.983050847458
Productivity
21028.410714285714
Business
7491.117647058823
Catalogs
4004.0
Medical
612.0


It seems as though Navigation has a higher number of reviews than any other category at 86090 reviews with Social Networking coming in at second with 71548 reviews.

We can do the same for the Googlle Play store however we will need to modify the genre names because they contain str characters like '+' and ','. To do this we can use the str.replace function.

In [21]:
table=freq_table(android_free, 1)
for category in table:
    total=0
    len_category=0
    for row in android_free:
        category_app=row[1]
        if category_app == category:
            installs=row[5]
            installs=installs.replace('+','')
            installs=installs.replace(',','')
            total=float(installs)
            len_category+=1
            average=total/len_category
    print(category)
    print(average)

ART_AND_DESIGN
1.7543859649122806
AUTO_AND_VEHICLES
1.2195121951219512
BEAUTY
1886.7924528301887
BOOKS_AND_REFERENCE
5.2631578947368425
BUSINESS
0.02457002457002457
COMICS
181.8181818181818
COMMUNICATION
174.21602787456445
DATING
606.060606060606
EDUCATION
9708.73786407767
ENTERTAINMENT
117647.05882352941
EVENTS
7.936507936507937
FINANCE
30.48780487804878
FOOD_AND_DRINK
0.09090909090909091
HEALTH_AND_FITNESS
366.3003663003663
HOUSE_AND_HOME
1.36986301369863
LIBRARIES_AND_DEMO
120481.92771084337
LIFESTYLE
28901.73410404624
GAME
116.0092807424594
FAMILY
0.059665871121718374
MEDICAL
3.194888178913738
SOCIAL
21186.4406779661
SHOPPING
5025.125628140703
PHOTOGRAPHY
38314.17624521073
SPORTS
0.0033222591362126247
TRAVEL_AND_LOCAL
4.830917874396135
TOOLS
133.33333333333334
PERSONALIZATION
1.7006802721088434
PRODUCTIVITY
0.028985507246376812
PARENTING
17.24137931034483
WEATHER
1408.4507042253522
VIDEO_PLAYERS
0.6289308176100629
NEWS_AND_MAGAZINES
403.2258064516129
MAPS_AND_NAVIGATION
40.32258064